In [48]:
import pandas as pd
import numpy as np
import math
import re
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [49]:
df = pd.read_excel("Raw.xlsx")
df.head()

,CustomerName,UserID,Item,ItemID,ItemDescription,Qty
0,AIRGAS SOUTH / GASO01,1,TH 80 02,1,WEAR HARD 1/8,10.0
1,AIRGAS SOUTH / GASO01,1,TE SUPER 500 03,2,SUPER 500 5/32,10.0
2,AIRGAS SOUTH / GASO01,1,TE SUPER 500 02,3,SUPER 500 1/8,10.0
3,AIRGAS SOUTH / GASO01,1,TF 630-16 02,4,E630-16 1/8 (17-4 PH),10.0
4,AIRGAS SOUTH / GASO01,1,TCU AB/06,5,"ALUMINUM BRONZE A2 1/8 X 36"" (ERCUAL-A2)",10.0


In [50]:
reader = Reader()
algo = SVD()

data = Dataset.load_from_df(df[['UserID', 'ItemID', 'Qty']], reader)

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    9563.59545742.09117005.70515742.49657458.35817102.44921406.6497
MAE (testset)     892.7487748.3449843.9742793.8046887.6591833.306355.4622 
Fit time          1.66    1.76    1.59    1.90    1.79    1.74    0.11    
Test time         0.12    0.04    0.04    0.06    0.04    0.06    0.03    


{'test_rmse': array([9563.59540884, 5742.09112065, 7005.70507138, 5742.49652698,
        7458.35805022]),
 'test_mae': array([892.74869521, 748.34487293, 843.97418728, 793.80455739,
        887.65907726]),
 'fit_time': (1.6590099334716797,
  1.7642583847045898,
  1.5930531024932861,
  1.9006397724151611,
  1.791468858718872),
 'test_time': (0.12167501449584961,
  0.041922569274902344,
  0.040927886962890625,
  0.05597996711730957,
  0.039926767349243164)}

We get a mean Root Mean Sqaure Error of ~0.70 which is more than good enough for our case.

In [95]:
# What UserID purchased in the past

UserID_1 = df[df['UserID'] == 10]
UserID_1.sort_values(by=['Qty'], ascending=False, inplace=True)
UserID_1[:10]

C:\Users\Shane Kim\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,CustomerName,UserID,Item,ItemID,ItemDescription,Qty
25273,JOHNS SALES & SERVICE INC.,10,TC 70S-6 104,2244,ER70S-6 .035 X 44 LB SPOOL,44880.0
24706,JOHNS SALES & SERVICE INC.,10,TC 70S-6 184,2277,ER70S-6 .045 X 44 LB SPOOL,15840.0
21092,JOHNS SALES & SERVICE INC.,10,TCC EAGLE-ARC 719 17,2245,"E71T-1C/1M, 9C/9M H8 .045 X 44 LB SPOOL",11880.0
21090,JOHNS SALES & SERVICE INC.,10,TE 7024 048,1995,"E7024 3/16 X 18"" (50 LB MASTER CTN)",10000.0
28145,JOHNS SALES & SERVICE INC.,10,TE 7018 02,391,E7018 1/8,8580.0
21667,JOHNS SALES & SERVICE INC.,10,TE 7024 038,1922,"E7024 5/32 X 18"" (50 LB MASTER CTN)",7100.0
25177,JOHNS SALES & SERVICE INC.,10,TE 7018 03,1017,E7018 5/32,6780.0
21932,JOHNS SALES & SERVICE INC.,10,TE 7018 02P,993,"1/8""",6000.0
21668,JOHNS SALES & SERVICE INC.,10,TE 7024 03,776,E7024 5/32,3720.0
24752,JOHNS SALES & SERVICE INC.,10,TE 7018 01,766,E7018 3/32,3000.0


In [97]:
# df that User didn't purchase
User_NotPurchased = df[df.UserID != 1]


# Insert drop_df


# Training the dataset
trainset = data.build_full_trainset()
algo.fit(trainset)

User_NotPurchased['Estimate_Score'] = User_NotPurchased['ItemID'].apply(lambda x: algo.predict(1, x).est)

User_NotPurchased = User_NotPurchased.drop(['ItemID','UserID'], axis = 1)

User_NotPurchased = User_NotPurchased.sort_values('Qty', ascending=False)
User_NotPurchased

C:\Users\Shane Kim\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,CustomerName,Item,ItemDescription,Qty,Estimate_Score
28617,"NEXAIR, LLC",TC 70S-6 093,ER70S-6 .035 X 33 LB SPOOL,599577.0,5
27820,Cal Tool / Helmets & 70S-6 093 & 123 Only,TC 70S-6 093,ER70S-6 .035 X 33 LB SPOOL,310464.0,5
25461,*WESTAIR GASES AND EQUIPMENT,WAG 70S-6 093,".035"" X 33 LBS Spool ER70S-6 (WAG W70S6035X33)",279972.0,5
28236,Airgas Group,TC 70S-6 093,ER70S-6 .035 X 33 LB SPOOL,228228.0,5
27777,IWS Gas & Supply of Texas,TC 70S-6 093,ER70S-6 .035 X 33 LB SPOOL,221133.0,5
28606,"NEXAIR, LLC",TC 70S-6 173,ER70S-6 .045 X 33 LB SPOOL,203511.0,5
27846,"WELSCO, INC.",TC 70S-6 093,ER70S-6 .035 X 33 LB SPOOL,185988.0,5
25268,*SIDNEY LEE WELDING SUPPLY,TC 70S-6 104,ER70S-6 .035 X 44 LB SPOOL,184800.0,5
27783,ROBERTS OXYGEN COMPANY INC,TC 70S-6 093,ER70S-6 .035 X 33 LB SPOOL,172110.3,5
25362,*Dupuy Oxygen and Supply Co.,TC 70S-6 104,ER70S-6 .035 X 44 LB SPOOL,167112.0,5


In [101]:
User_NotPurchased 

AttributeError: Cannot access callable attribute 'groupby' of 'DataFrameGroupBy' objects, try using the 'apply' method

In [ ]:
benchmark = round(df['Qty'].quantile(0.7),0)
drop_item_list = df[df['Qty'] < benchmark].index

User_NotPurchased = User_NotPurchased.copy()
User_NotPurchased = User_NotPurchased.reset_index()
User_NotPurchased = User_NotPurchased[~User_NotPurchased['ItemID'].isin(drop_item_list)]



